In [1]:
!pip install tensorflow==2.5.0 tensorflow-gpu==2.5.0 opencv-python matplotlib

In [2]:
!c:\users\diego\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip

In [3]:
# Import dependencies 
import cv2 
import os 
import random 
import numpy as np 
from matplotlib import pyplot as plt

In [4]:
# Import tf depen - functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten 
import tensorflow as tf

In [16]:
# Set GPU Growth 
# Avoid OOM errors by seting GPU Memory Consumption Growth 
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

[]


In [19]:
# Setup paths 
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

# Create folders 
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [24]:
# Decompress dataset 
# !tar -xf lfw.tgz

# Move images to following repository data/negative 
for directory in os.listdir('lfw'): 
    for file in os.listdir(os.path.join('lfw', directory)): 
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

In [62]:
# Import uuid lib to generate unique image names 
import uuid 

In [71]:
# Establish collection with to the webcam 
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW) 

while cap.isOpened():
    # take frame
    ret, frame = cap.read()
    
    #resize frames 
    frame = frame[120:120+250, 200:200+250, :]
    
    # print frame 
    cv2.imshow('Image Collection', frame )
    
    # a key collect anchor 
    if cv2.waitKey(1) & 0XFF == ord('a'):
        
        imgname = os.path.join(ANC_PATH, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(imgname, frame)
    # p key collect positives 
    if cv2.waitKey(1) & 0XFF == ord('p'):
        imgname = os.path.join(POS_PATH, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(imgname, frame)
    # q key closes everything 
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
# destroy windows         
cap.release()
cv2.destroyAllWindows()

In [72]:
# Pre proccessing pipeline 
anchor = tf.data.Dataset.list_files(ANC_PATH + '\*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH + '\*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH + '\*.jpg').take(300)

In [75]:
# preproccesing function 
def preprocess(file_path): 
    # read image for filepath 
    byte_img = tf.io.read_file(file_path)
    # load in image 
    img = tf.io.decode_jpeg(byte_img)
    # PREPROCESS : resize 100x100x3 - scale image btw 0 and 1  
    img = tf.image.resize(img, (100,100))
    img = img / 255 
    return img 

In [89]:
# Create labelled dataset 
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [ ]:
samples = data.as_numpy_iterator()

In [422]:
ex = samples.next()

In [424]:
def preprocess_twin(input_img, validation_img, label):
    return (preprocess(input_img), preprocess(validation_img), label)

In [431]:
# Build dataloader pipeline 
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size = 1024)

In [433]:
# Training partition 
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

# Testing partition 
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)